# Lesson 1 Inkling
- The first Inkling used is from the [house-energy](https://github.com/microsoft/microsoft-bonsai-api/blob/main/Python/samples/house-energy/machine_teacher.ink) folder in the *microsoft-bonsai-api* repo.

- It has a goal based curriculum that minimizes the difference between the thermostat setting and the temperature inside the building.

- It has one lesson that varies the initial temperature inside between 18 and 30 and sets the other values to the default values that are defined in the [simulator code](https://github.com/microsoft/microsoft-bonsai-api/blob/main/Python/samples/house-energy/sim/house_simulator.py) 

```python
inkling "2.0"
using Number
using Goal
using Math

const MaxDeviation = 2
const MaxIterations = 288

type SimState {
    Tset: Number.Float32,
    Tin: Number.Float32,
    Tout: Number.Float32,
}

type SimAction {
    hvacON: Number.Int8 <off=0, on=1,>
}

type SimConfig {
    K: Number.Float32, # Thermal conductivity
    C: Number.Float32, # Thermal Capacity
    Qhvac: Number.Float32, # Heat Flux
    Tin_initial: number, # C
    schedule_index: Number.Int8, # 1 - fixed, 2 - randomized
    number_of_days: number,
    timestep: number, # Min
    max_iterations: number, # Alters schedule generation
}

function TempDiff(Tin:number, Tset:number) {
    return Math.Abs(Tin - Tset)
}


graph (input: SimState): SimAction {
    concept SmartHouse(input): SimAction {
        curriculum {
            source simulator (Action: SimAction, Config: SimConfig): SimState {
                package "HouseEnergy"
            }
            goal (State: SimState) {
                minimize `Temp Deviation`:
                    TempDiff(State.Tin, State.Tset) in Goal.RangeBelow(MaxDeviation)
            }

            training {
                EpisodeIterationLimit: MaxIterations
            }

            lesson `Lesson 1` {
                scenario {
                    K: 0.5,
                    C: 0.3,
                    Qhvac: 9,
                    Tin_initial: number<18 .. 30>,
                    schedule_index: 2,
                    number_of_days: 1,
                    timestep: 5,
                    max_iterations: MaxIterations
                }
            }
        }
    }
}
    
```

## The following calculates the root mean square of the Lesson 1 Inkling

In [44]:
import pandas as pd
import json

# Parse lesson 1 csv
data = pd.read_csv('../csv/lesson1.csv')
strdata = data['SimState_s'].apply(json.loads)
strlst = list(strdata)
strjson = json.dumps(strlst)
data = data.join(pd.read_json(strjson))

rootMeanSquare = ((data["Tset"] - data["Tin"]) ** 2).mean() ** .5
print(rootMeanSquare)


0.9386015491502107


## The following calculates the root mean square of the random baseline:

In [4]:
import pandas as pd
import json
data = pd.read_json('../csv/random.json')
rootMeanSquare = ((data["Tset"] - data["Tin"]) ** 2).mean() ** .5
print(rootMeanSquare)

3.4328605246987633
